### This notebook will be mainly used for the Coursera capstone project

In [1]:
# Import dependencies

import pandas as pd
import numpy as np
import time

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

### Web-Scraping Wikipedia

In [2]:
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
table = soup.find_all('table')[0]
lists = pd.read_html(str(table))

df = lists[0]

#### Initial Dataframe

In [3]:
print(f'Initial dataset size: {df.shape}')
df.head()

Initial dataset size: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Ignoring cells with a borough that is Not assigned.

In [4]:
indexBorough = df[df['Borough'] == 'Not assigned'].index
df.drop(indexBorough, inplace=True)
print(f'Updated dataset size: {df.shape}')
df.head(10)

Updated dataset size: (103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
NA_neighborhoods = df[df['Neighbourhood'] == 'Not assigned']

# How many neighborhoods are not assigned?

print(f'There are {len(NA_neighborhoods)} neighborhoods that are not assigned, but belong to a borough.')

There are 0 neighborhoods that are not assigned, but belong to a borough.


#### Clean the dataframe

In [6]:
# Dropping rows messed with the index. Time to unmess it up!
df.reset_index(drop=True).head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### How many rows does this dataframe have?

In [7]:
num_rows = df.shape[0]
num_cols = df.shape[1]

print(f'There are {num_rows} rows and {num_cols} columns.')

There are 103 rows and 3 columns.


### Getting the latitude and the longitude coordinates of each neighborhood

In [8]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
merged_df = pd.merge(df, df_data_2, on='Postal Code', how='left')

num_boroughs = len(merged_df['Borough'].unique())
num_neigh = merged_df.shape[0]
print(f'')
print(f'The dataframe has {num_boroughs} boroughs and {num_neigh} neighborhoods.')
print(f'')
merged_df.head(12)


The dataframe has 10 boroughs and 103 neighborhoods.



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [10]:

# Map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

### Getting geographical coordinates of Toronto, Canada

In [11]:
address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Map of Toronto with markers for each neighborhood

In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [13]:
# The code was removed by Watson Studio for sharing.

#### Getting the latitude and longitude of Church and Wellesley

In [14]:
neighborhood_name = merged_df.loc[99, 'Neighbourhood']
neighborhood_lat = merged_df.loc[99, 'Latitude']
neighborhood_long = merged_df.loc[99, 'Longitude']
print(f'')
print(f'{neighborhood_name} has a latitude of {neighborhood_lat} and longitude of {neighborhood_long}.')


Church and Wellesley has a latitude of 43.6658599 and longitude of -79.38315990000001.


#### Getting 100 venues within a 500 meter radius of Church and Wellesley

In [15]:
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    limit)

results = requests.get(url).json()

#### Create a function to extract information from the *items* key

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the *results* JSON and structure it into a **dataframe**.

In [17]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Storm Crow Manor,Theme Restaurant,43.666840,-79.381593
1,DanceLifeX Centre,Dance Studio,43.666956,-79.385297
2,Smith,Breakfast Spot,43.666927,-79.381421
3,Ho's Team Barber Shop,Salon / Barbershop,43.665630,-79.381359
4,Glad Day Bookshop,Bookstore,43.665271,-79.380785
5,Barbara Hall Park,Park,43.666879,-79.381068
6,Fabarnak,Restaurant,43.666377,-79.380964
7,Bar Volo,Beer Bar,43.665462,-79.385692
8,The Alley,Bubble Tea Shop,43.665922,-79.385567
9,Sansotei Ramen 三草亭,Ramen Restaurant,43.666735,-79.385353


#### How many venues did the Foursquare API call actually return?

In [18]:
print(f'{nearby_venues.shape[0]} venues near {neighborhood_name} were returned by the Foursquare API call.')

76 venues near Church and Wellesley were returned by the Foursquare API call.


#### Let's create a function to repeat the process to all neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Let us run this function on each neighborhood and create a new dataframe called toronto_venues

In [20]:
%%capture
toronto_venues = getNearbyVenues(names=merged_df['Neighbourhood'],
                                   latitudes=merged_df['Latitude'],
                                   longitudes=merged_df['Longitude']
                                  )

#### How large is the resulting dataframe?

In [21]:
print(f'There are {toronto_venues.shape[0]} rows and {toronto_venues.shape[1]} columns in the dataframe below')
toronto_venues.head()

There are 2146 rows and 7 columns in the dataframe below


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### How many venues were returned for each neighborhood?

In [22]:
grouped_by_neighborhood = toronto_venues.groupby('Neighborhood').count()
grouped_by_neighborhood.drop(grouped_by_neighborhood.iloc[:, 0:5], inplace=True, axis = 1)
grouped_by_neighborhood.rename(columns = {'Venue Category': 'Venue Count'}, inplace=True)
grouped_by_neighborhood

,Venue Count
Neighborhood,
Agincourt,4
"Alderwood, Long Branch",6
"Bathurst Manor, Wilson Heights, Downsview North",22
Bayview Village,4
"Bedford Park, Lawrence Manor East",24
Berczy Park,58
"Birch Cliff, Cliffside West",4
"Brockton, Parkdale Village, Exhibition Place",22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16


#### How many unique categories can be curated from all the returned venues?

In [23]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 unique categories.


### Analyzing Each Neighborhood in Toronto

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### What is the size of this new dataframe?

In [25]:
print(f'This dataframe has {toronto_onehot.shape[0]} rows and {toronto_onehot.shape[1]} columns.')

This dataframe has 2146 rows and 264 columns.


#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### What is the new size?

In [27]:
print(f'This dataframe has {toronto_grouped.shape[0]} rows and {toronto_grouped.shape[1]} columns.')

This dataframe has 96 rows and 264 columns.


### Printing each neighborhood with its 5 most common venues.

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2     Coffee Shop  0.17
3             Pub  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0                Bank  0.09
1         Coffee Shop  0.09
2  Chinese Restaurant  0.05
3         Gas Station  0.05
4       Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.08
1               Restaurant  0.08
2              Coffee Shop

### Let's put this all into a pandas dataframe.

First, let's write a function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now, what are the top venues for each neighborhood?

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Pub,Coffee Shop,Airport Food Court,Airport Lounge,Eastern European Restaurant,Dumpling Restaurant,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Fried Chicken Joint,Supermarket,Ice Cream Shop,Sushi Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Restaurant
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,Juice Bar,Butcher,Pub,Café,Fast Food Restaurant,Sushi Restaurant
5,Berczy Park,Coffee Shop,Restaurant,Café,Farmers Market,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Eastern European Restaurant
6,"Birch Cliff, Cliffside West",Café,College Stadium,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
8,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Coffee Shop,Rental Car Location,Plane,Bar,Harbor / Marina,Boat or Ferry,Airport Food Court


### Cluster Neighborhoods

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
# Comment/uncomment below in order to get Cluster Labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].replace(np.nan,0)
toronto_merged = toronto_merged.astype({"Cluster Labels": int})
toronto_merged.head() # Check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Portuguese Restaurant,French Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Yoga Studio,Ice Cream Shop,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Gym,Sandwich Place,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Portuguese Restaurant,French Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store
2,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Yoga Studio,Ice Cream Shop,Spa
3,North York,0,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Vietnamese Restaurant,Accessories Store
4,Downtown Toronto,0,Coffee Shop,Diner,Gym,Sandwich Place,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,North York,0,Gym,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Supermarket,Clothing Store,Italian Restaurant,Bike Shop,Discount Store
8,East York,0,Pizza Place,Breakfast Spot,Intersection,Athletics & Sports,Café,Gastropub,Bank,Pharmacy,Pet Store,Gym / Fitness Center
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Hotel,Plaza
10,North York,0,Asian Restaurant,Metro Station,Pub,Japanese Restaurant,Women's Store,Diner,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
11,Etobicoke,0,Jewelry Store,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 2

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,1,Pizza Place,Playground,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Distribution Center
50,North York,1,Gym,Pizza Place,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


#### Cluster 3

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
21,York,2,Park,Pool,Women's Store,Gluten-free Restaurant,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
35,East York,2,Park,Coffee Shop,Convenience Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
49,North York,2,Park,Bakery,Construction & Landscaping,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dance Studio,Distribution Center
52,North York,2,Park,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
64,York,2,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
66,North York,2,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
85,Scarborough,2,Park,Playground,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
91,Downtown Toronto,2,Park,Playground,Trail,Drugstore,Donut Shop,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center,Cupcake Shop
98,Etobicoke,2,River,Pool,Park,College Rec Center,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3,Baseball Field,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
101,Etobicoke,3,Baseball Field,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 5

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Fast Food Restaurant,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
